In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


from astropy.table import Table, vstack
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
import glob
#from muonpipe import usefull_func
import seaborn as sns
from ctapipe.image import tailcuts_clean
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm
from datetime import datetime

filename = '/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry
summer_refractive_index = 1+0.00020409512542851616
winter_refractive_index = 1+0.00020953240736654166

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397

ring_containment_cut = 0.99
ring_completeness_cut = 0.99
min_radius_cut = 0.95
max_radius_cut = 1.15


In [ ]:
muon_files = glob.glob('/Users/vdk/Documents/DocumentsVadymMacBookWork/all_muon_fits/low_nsb_filter/*')
df_files = [
    pd.read_csv(muon_file, na_values=['NA', '?']).query(
        '(muon_efficiency < @muon_efficiency_cut) & '
        '(size_outside < @size_outside_cut) & '
        '(ring_completeness > @ring_completeness_cut) & '
        '(ring_containment > @ring_containment_cut) & '
        'ring_radius.between(@min_radius_cut, @max_radius_cut)'
    )
    for muon_file in sorted(muon_files)[10:]
]

df_good_data_real = pd.concat(df_files, ignore_index=True)
df_good_data_real

In [ ]:
# Check the minimum and maximum values of appropriate columns in df_good_data_real
print(f"Minimum Ring Completeness in df_good_data_real: {df_good_data_real['ring_completeness'].min()}")
print(f"Maximum Ring Completeness in df_good_data_real: {df_good_data_real['ring_completeness'].max()}")
print(f"Minimum Ring Radius in df_good_data_real: {df_good_data_real['ring_radius'].min()}")
print(f"Maximum Ring Radius in df_good_data_real: {df_good_data_real['ring_radius'].max()}")
print(f"Minimum Muon Efficiency in df_good_data_real: {df_good_data_real['muon_efficiency'].min()}")
print(f"Maximum Muon Efficiency in df_good_data_real: {df_good_data_real['muon_efficiency'].max()}")
print(f"Minimum Size Outside in df_good_data_real: {df_good_data_real['size_outside'].min()}")
print(f"Maximum Size Outside in df_good_data_real: {df_good_data_real['size_outside'].max()}")

# Convert event_time to human-readable format
min_event_time = datetime.fromtimestamp(df_good_data_real['event_time'].min())
max_event_time = datetime.fromtimestamp(df_good_data_real['event_time'].max())
print(f"Minimum Event Time in df_good_data_real: {min_event_time}")
print(f"Maximum Event Time in df_good_data_real: {max_event_time}")

In [ ]:
#listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/nsb_play/nsb_tune_2024_year_no_outliers/tables/*')

listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')

#listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_no_outliers_additional/tables/*')

#listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/tables/*')

listdir = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')

listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)    
    dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_2024_tune = df[df['good_ring'] & 
                      (df['muon_efficiency'] < muon_efficiency_cut) & 
                      (df['size_outside'] < size_outside_cut) &
                      df['ring_radius'].between(min_radius_cut, max_radius_cut) &
                      df['ring_completeness'].between(ring_completeness_cut, 1)]
df_sim_2024_tune

In [ ]:
# Check the minimum and maximum values of appropriate columns in df_sim_2024_tune
print(f"Minimum Ring Completeness in df_sim_2024_tune: {df_sim_2024_tune['ring_completeness'].min()}")
print(f"Maximum Ring Completeness in df_sim_2024_tune: {df_sim_2024_tune['ring_completeness'].max()}")
print(f"Minimum Ring Radius in df_sim_2024_tune: {df_sim_2024_tune['ring_radius'].min()}")
print(f"Maximum Ring Radius in df_sim_2024_tune: {df_sim_2024_tune['ring_radius'].max()}")
print(f"Minimum Muon Efficiency in df_sim_2024_tune: {df_sim_2024_tune['muon_efficiency'].min()}")
print(f"Maximum Muon Efficiency in df_sim_2024_tune: {df_sim_2024_tune['muon_efficiency'].max()}")
print(f"Minimum Size Outside in df_sim_2024_tune: {df_sim_2024_tune['size_outside'].min()}")
print(f"Maximum Size Outside in df_sim_2024_tune: {df_sim_2024_tune['size_outside'].max()}")

In [ ]:
df_good_data_real_2024 = df_good_data_real[
    (df_good_data_real['event_time'] >= start_date_2024) & 
    (df_good_data_real['event_time'] <= end_date_2024)
]

# Check the minimum and maximum values of appropriate columns in df_good_data_real_2024
print(f"Minimum Ring Completeness in df_good_data_real_2024: {df_good_data_real_2024['ring_completeness'].min()}")
print(f"Maximum Ring Completeness in df_good_data_real_2024: {df_good_data_real_2024['ring_completeness'].max()}")
print(f"Minimum Ring Radius in df_good_data_real_2024: {df_good_data_real_2024['ring_radius'].min()}")
print(f"Maximum Ring Radius in df_good_data_real_2024: {df_good_data_real_2024['ring_radius'].max()}")
print(f"Minimum Muon Efficiency in df_good_data_real_2024: {df_good_data_real_2024['muon_efficiency'].min()}")
print(f"Maximum Muon Efficiency in df_good_data_real_2024: {df_good_data_real_2024['muon_efficiency'].max()}")
print(f"Minimum Size Outside in df_good_data_real_2024: {df_good_data_real_2024['size_outside'].min()}")
print(f"Maximum Size Outside in df_good_data_real_2024: {df_good_data_real_2024['size_outside'].max()}")

# Convert event_time to human-readable format
min_event_time_2024 = datetime.fromtimestamp(df_good_data_real_2024['event_time'].min())
max_event_time_2024 = datetime.fromtimestamp(df_good_data_real_2024['event_time'].max())
print(f"Minimum Event Time in df_good_data_real_2024: {min_event_time_2024}")
print(f"Maximum Event Time in df_good_data_real_2024: {max_event_time_2024}")

In [ ]:
year_value = 2024
data_mc = df_sim_2024_tune
data_run = df_good_data_real_2024

bins = np.linspace(min_radius_cut, max_radius_cut, 10)

# Generate x values for the regression lines
x_values = np.linspace(bins.min(), bins.max(), 300)
x_values_with_const = sm.add_constant(x_values)

# Bin the data for MC
data_mc['binned_radius'] = pd.cut(data_mc['ring_radius'], bins, include_lowest=True, labels=False)
binned_mc = data_mc.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_mc['ring_size'] = binned_mc['ring_size']
binned_mc['constant'] = 1
model_mc = sm.OLS(binned_mc['ring_size'], binned_mc[['constant', 'ring_radius']]).fit()
y_values_mc = model_mc.predict(x_values_with_const)
prediction_mc = model_mc.get_prediction(x_values_with_const)
conf_int_mc = prediction_mc.conf_int()

# Bin the data for Run
data_run['binned_radius'] = pd.cut(data_run['ring_radius'], bins, include_lowest=True, labels=False)
binned_run = data_run.groupby('binned_radius').agg({
    'ring_radius': 'mean',
    'ring_size': 'mean'
})
binned_run['constant'] = 1
model_run = sm.OLS(binned_run['ring_size'], binned_run[['constant', 'ring_radius']]).fit()
y_values_run = model_run.predict(x_values_with_const)
prediction_run = model_run.get_prediction(x_values_with_const)
conf_int_run = prediction_run.conf_int()


# Plotting
plt.figure(figsize=(12, 12))

# Plotting MC data
plt.scatter(binned_mc['ring_radius'], binned_mc['ring_size'], color='blue', label='Simulation optical efficiency tuned to 2024 year ', s=50)
plt.plot(x_values, y_values_mc, color='blue', linestyle='--', label='MC regression line')
plt.fill_between(x_values, conf_int_mc[:, 0], conf_int_mc[:, 1], color='blue', alpha=0.2)

# Plotting Data Run
plt.scatter(binned_run['ring_radius'], binned_run['ring_size'], color='orange', label=f'Data for {year_value} year', s=50)

plt.plot(x_values, y_values_run, color='orange', linestyle='--', label='Data regression line')
plt.fill_between(x_values, conf_int_run[:, 0], conf_int_run[:, 1], color='orange', alpha=0.2)


# Adding labels and legend
plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon ring size [p.e.]')
plt.legend()
plt.ylim(1800,3000)
#plt.savefig(f'/Users/vdk/muons2024/images/ring_size_check/full_sample_2024NSBtune_size_vs_radius_{year_value}year_comp{ring_completeness_cut}.pdf', dpi=200, format='pdf', bbox_inches='tight')

In [ ]:
list(binned_mc['ring_size'])

In [ ]:
list(binned_run['ring_size'])

In [ ]:
mse = np.mean((binned_mc['ring_size'] - binned_run['ring_size']) ** 2)
print(f"Mean Squared Error: {mse}") 

In [ ]:
mean_mc = np.mean(binned_mc['ring_size'])
mean_data = np.mean(binned_run['ring_size'])
overall_mean = (mean_mc + mean_data) / 2
print(f"Mean of binned_mc: {mean_mc:.2f}")
print(f"Mean of binned_data: {mean_data:.2f}")
print(f"Overall Mean: {overall_mean:.2f}")
print(f"Root Mean Squared Error: {mse**0.5:.2f}") 
print(f"RMSE persentage: {mse**0.5 / overall_mean * 100:.2f}%")  

In [ ]:
# Calculate standard deviations
std_mc = np.std(binned_mc['ring_size'], ddof=1)
std_data = np.std(binned_run['ring_size'], ddof=1)
average_std = (std_mc + std_data) / 2
print(f"Standard Deviation of binned_mc: {std_mc}")
print(f"Standard Deviation of binned_data: {std_data}")
print(f"Average Standard Deviation: {average_std}")